# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s):
###### Python version:
###### Submission deadline: 1pm, Fri 5 Apr 2019

In [32]:
# This function should open a data file in csv, and transform it into a usable format
import csv
def preprocess(filename, header):
    dataset = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile, header)
        for row in reader:
            dataset.append(row)
    return dataset



In [33]:
def train(dataset, header, classlabels):
    model = []
    smoothing = []
    classes = {}
    count = 0
    for index in range(len(dataset)):
        count += 1;
        if dataset[index]['class'] in classes:
            classes[dataset[index]['class']] += 1
        else:
            classes[dataset[index]['class']] = 1
    classes['count'] = count
    model.append(classes)
    
    #Laplace smoothing
    j = 0
    while j < len(header)-1:
        attributes = {}
        count = 0
        for index in range(len(dataset)):
            if dataset[index][header[j]] not in attributes:
                count += 1
                attributes[dataset[index][header[j]]] = 1
        j+=1
        attributes['count'] = count
        smoothing.append(attributes)
    i = 0
    
    while i < len(classlabels):
        current_class = classlabels[i]
        j = 0
        class_attributes = []
        while j < len(header)-1:
            attributes = smoothing[j].copy()
            for index in range(len(dataset)):
                if dataset[index]['class'] == current_class:
                    attributes['count'] += 1
                    attributes[dataset[index].get(header[j])] += 1
            class_attributes.append(attributes)
            j+=1
        model.append(class_attributes)
        i+=1
    return model

In [34]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, dataset, classlabels, header):
    predicted_classes = []
    highest_probability = 0
    probability = 0
    for index in range(len(dataset)):
        row = dataset[index]
        highest_probability = 0
        probability = 0
        predicted_label = ''
        for label in classlabels:
            probability = model[0][label]/model[0]['count']
            for attribute in range(len(row)-1):
                probability *= model[classlabels.index(label)+1][attribute][row.get(header[attribute])]/model[classlabels.index(label)+1][attribute]['count']
            if probability > highest_probability:
                highest_probability = probability
                predicted_label = label
        predicted_classes.append(predicted_label)
    return predicted_classes

In [114]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(predicted_classes, dataset):
    correct = 0
    wrong = 0
    count = 0
    for index in range(len(dataset)):
        row = dataset[index]
        if predicted_classes[index] == row['class']:
            correct += 1
        else:
            wrong += 1
        count += 1
    print("Accuracy is", correct/count*100,"% with", correct,"correct predictions and", wrong, "wrong predictions" )


In [172]:
import math
def info_gain(dataset,classlabels,headers):
    ig = {} #dict for information gain
    HR = 0 #total HR
    M = 0  # for mean infor
    total_value = 0 #for total count of each class in each attribute
    V =[] 
    W=[] #store all the values in a list of a list
    P = [] #store the probobility of each attribute
    
    #Get HR
    for j in range(len(classlabels)):
        HR += -((model[0][classlabels[j]]/model[0]['count'])*math.log2(model[0][classlabels[j]]/model[0]['count']))
    print(HR)
    
    #we want to get 6 ig so use for loop to process 6 times
    for k in range(len(header)-1):
        total_value = 0
        #put all the values into a list
        for i in range(len(model)-1):
            values = list(model[i+1][k].values())
            V.append(values)
        W.append(V)
        V=[]
    print(W)
    
    #get entrypy for each small attribute for all the large attributes and put them into a list
    #get the probobility of each class inside the large attribute and put them into a list
    for i in range(len(W)):
        value = []
        H = []
        P = []
        for j in range(len(W[i])):
            last_count = 0
            k = 0
            while k < len(W[i]):
                print(W[i][k][j])
                total_value += W[i][k][j]
                last_count += W[i][k][-1]
                k+=1
            print('total value',total_value)
            value.append(total_value)
            H_ind = 0
            if (j<len(W[i][j])-1):  
                for k in range(len(W[i])):
                    if ((W[i][k][j]-1)>1):
                        H_ind +=  -(((W[i][k][j]-1)/(total_value-4)))*(math.log2((W[i][k][j]-1)/(total_value-4)))
                H.append(H_ind)
                P.append(total_value/last_count)
            total_value = 0
        M = 0
        # get mean then get information gain for each attribute toward the classifer 
        for j in range(len(P)):
            M+= P[j]*H[j]
        ig[header[i]] = HR - M
    print(ig)
    return 
info_gain(dataset,classlabels,header)

1.2057409700121753
[[[361, 325, 269, 259, 1214], [73, 109, 116, 90, 388], [1, 1, 24, 47, 73], [1, 1, 27, 40, 69]], [[361, 315, 269, 269, 1214], [73, 106, 116, 93, 388], [1, 1, 24, 47, 73], [1, 14, 27, 27, 69]], [[327, 301, 293, 293, 1214], [82, 100, 103, 103, 388], [16, 19, 19, 19, 73], [11, 16, 21, 21, 69]], [[577, 313, 323, 1213], [1, 199, 187, 387], [1, 37, 34, 72], [1, 31, 36, 68]], [[451, 393, 369, 1213], [106, 136, 145, 387], [22, 25, 25, 72], [1, 26, 41, 68]], [[577, 358, 278, 1213], [1, 181, 205, 387], [1, 40, 31, 72], [1, 1, 66, 68]]]
{'buying': 0.09644896916961399, 'maint': 0.07370394692148596, 'doors': 0.004485716626632108, 'persons': 0.2196629633399082, 'lug_boot': 0.030008141247605424, 'safety': 0.262184356554264}


# 

In [116]:
header = 'buying','maint','doors','persons','lug_boot','safety','class'
dataset = preprocess('car.csv',header)
classlabels = 'unacc', 'acc', 'good', 'vgood'
print(classlabels)
model = train(dataset, header, classlabels)
print(model)
predicted_classes = predict(model, dataset, classlabels, header)
evaluate(predicted_classes, dataset)

('unacc', 'acc', 'good', 'vgood')
[{'unacc': 1210, 'acc': 384, 'vgood': 65, 'good': 69, 'count': 1728}, [{'vhigh': 361, 'high': 325, 'med': 269, 'low': 259, 'count': 1214}, {'vhigh': 361, 'high': 315, 'med': 269, 'low': 269, 'count': 1214}, {'2': 327, '3': 301, '4': 293, '5more': 293, 'count': 1214}, {'2': 577, '4': 313, 'more': 323, 'count': 1213}, {'small': 451, 'med': 393, 'big': 369, 'count': 1213}, {'low': 577, 'med': 358, 'high': 278, 'count': 1213}], [{'vhigh': 73, 'high': 109, 'med': 116, 'low': 90, 'count': 388}, {'vhigh': 73, 'high': 106, 'med': 116, 'low': 93, 'count': 388}, {'2': 82, '3': 100, '4': 103, '5more': 103, 'count': 388}, {'2': 1, '4': 199, 'more': 187, 'count': 387}, {'small': 106, 'med': 136, 'big': 145, 'count': 387}, {'low': 1, 'med': 181, 'high': 205, 'count': 387}], [{'vhigh': 1, 'high': 1, 'med': 24, 'low': 47, 'count': 73}, {'vhigh': 1, 'high': 1, 'med': 24, 'low': 47, 'count': 73}, {'2': 16, '3': 19, '4': 19, '5more': 19, 'count': 73}, {'2': 1, '4': 37, '

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

1.Yes, when the information gain of each attribute is higher than others, then it means that it can be a good choice.


MISS VALUE
1.can take question mark as a value which the attribute will have 3 attributes which are T,F,? just treat it as a single value
2.still have question marks in the set but - the question marks at the end. 1/3-1
3.it is not nessary to 
Question 2: 
evidence of independance  calculate the attribute one and two's information gain. so not calculating the class but the infomation gain between two attributes

for all the attributes get the information gain and put them in a list
information gain:
just culculate 4 of them.


4,5(add_k)
